In [1]:
from core.multi_case_modeller import MultiCaseModeller
from sixgill.definitions import ModelComponents, Parameters
from sixgill.pipesim import Model, Units
from core import ExcelInputError

In [2]:
m = r"C:\Users\IDM252577\Desktop\Python Projects\PIPSIM\Models\O-Max_S-HP_trail.pips"

In [3]:
model = Model.open(m)

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 18688
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.


In [4]:
model.tasks.networksimulation.validate()

[]